In [6]:
import QuantLib as ql

In [32]:
#chris, haonan project#
class PricerBase:

    def __init__(self, config):
        self.risk_free_rate = config['risk_free_rate']
        self.libor_rate = config['libor_rate']
        self.calculation_date = config['calculation_date']
        if config['forward'] != None:
            self.forward = int(config['forward'][:-1])
        else:
            self.forward = 0
        self.forward_dt = config['forward'][-1]
        self.maturity = int(config['maturity'][:-1])
        self.fixed_tenor = int(config['fixed_tenor'][:-1])
        self.float_tenor = int(config['float_tenor'][:-1])
        self.maturity_dt = config['maturity'][-1]
        self.fixed_tenor_dt = config['fixed_tenor'][-1]
        self.float_tenor_dt = config['float_tenor'][-1]
    
    def dateType(self, date_type):
        if date_type == 'y':
            return ql.Years
        elif date_type == 'm':
            return ql.Months
        elif date_type == 'd':
            return ql.Days

    def discountCurve(self):
        day_count = ql.Actual365Fixed()
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.risk_free_rate, day_count))
        return discount_curve

    def liborIndex(self):
        day_count = ql.Actual365Fixed()
        libor_curve = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.libor_rate, day_count))
        libor_index = ql.Euribor3M(libor_curve)  
        libor_index = ql.USDLibor(ql.Period(self.float_tenor, self.dateType(self.float_tenor_dt)), libor_curve)
        return libor_index

    def fixFloatLegSchedule(self):
        calendar = ql.UnitedKingdom()
        settle_date = calendar.advance(self.calculation_date, self.forward, self.dateType(self.forward_dt))
        maturity_date = calendar.advance(settle_date, self.maturity, self.dateType(self.maturity_dt))
        fixed_leg_tenor = ql.Period(self.fixed_tenor, self.dateType(self.fixed_tenor_dt))
        fixed_schedule = ql.Schedule(settle_date, maturity_date, 
                                    fixed_leg_tenor, calendar,
                                    ql.ModifiedFollowing, ql.ModifiedFollowing,
                                    ql.DateGeneration.Forward, False)
        float_leg_tenor = ql.Period(self.float_tenor, self.dateType(self.float_tenor_dt))
        float_schedule = ql.Schedule (settle_date, maturity_date, 
                                    float_leg_tenor, calendar,
                                    ql.ModifiedFollowing, ql.ModifiedFollowing,
                                    ql.DateGeneration.Forward, False)
        return fixed_schedule, float_schedule

    def price(self, notional, fixed_rate, float_spread):
        return 0

In [33]:
class VanillaSwap(PricerBase):

    def __init__(self, config):
        super().__init__(config)

    def Pricer(self, config):
        notional = config['notional']
        fixed_rate = config['fixed_rate']
        float_spread = config['float_spread']
        fixed_leg_daycount = ql.Actual365Fixed()
        float_leg_daycount = ql.Actual365Fixed()
        discount_curve = self.discountCurve()
        libor_index = self.liborIndex()
        fixed_schedule, float_schedule = self.fixFloatLegSchedule()
        ir_swap = ql.VanillaSwap(ql.VanillaSwap.Payer, notional, fixed_schedule, 
                    fixed_rate, fixed_leg_daycount, float_schedule,
                    libor_index, float_spread, float_leg_daycount )
        swap_engine = ql.DiscountingSwapEngine(discount_curve)
        
        ir_swap.setPricingEngine(swap_engine)
        return ir_swap

In [34]:
my_config = {'risk_free_rate':0.01,'libor_rate':0.02,'calculation_date':ql.Date(31, 12, 2021),'forward':'5d','maturity':'10y',
            'fixed_tenor':'6m','float_tenor':'3m','notional':10e6,'fixed_rate':0.025,'float_spread':0.004}

In [35]:
vanilla = VanillaSwap(my_config)
price = vanilla.Pricer(my_config)

In [36]:
print("Fixed Leg Cashflows:")
for i, cf in enumerate(price.leg(0)):
    print("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

Fixed Leg Cashflows:
 1    July 11th, 2022      124657.53
 2    January 10th, 2023   125342.47
 3    July 10th, 2023      123972.60
 4    January 10th, 2024   126027.40
 5    July 10th, 2024      124657.53
 6    January 10th, 2025   126027.40
 7    July 10th, 2025      123972.60
 8    January 12th, 2026   127397.26
 9    July 10th, 2026      122602.74
10    January 11th, 2027   126712.33
11    July 12th, 2027      124657.53
12    January 10th, 2028   124657.53
13    July 10th, 2028      124657.53
14    January 10th, 2029   126027.40
15    July 10th, 2029      123972.60
16    January 10th, 2030   126027.40
17    July 10th, 2030      123972.60
18    January 10th, 2031   126027.40
19    July 10th, 2031      123972.60
20    January 12th, 2032   127397.26


In [37]:
for i, cf in enumerate(price.leg(1)):
    print(i+1, cf.date(), cf.amount())

1 April 11th, 2022 59275.37855344458
2 July 11th, 2022 59275.37855344458
3 October 10th, 2022 59275.37855344458
4 January 10th, 2023 59928.123104113496
5 April 11th, 2023 59275.37855344239
6 July 10th, 2023 58622.66376390512
7 October 10th, 2023 59928.123104113496
8 January 10th, 2024 59928.12310411568
9 April 10th, 2024 59275.37855344458
10 July 10th, 2024 59275.37855344458
11 October 10th, 2024 59928.123104113496
12 January 10th, 2025 59928.12310411568
13 April 10th, 2025 58622.663763902936
14 July 10th, 2025 59275.37855344458
15 October 10th, 2025 59928.123104113496
16 January 12th, 2026 61233.70149536812
17 April 10th, 2026 57317.323461694854
18 July 10th, 2026 59275.37855344458
19 October 12th, 2026 61233.70149536592
20 January 11th, 2027 59275.37855344677
21 April 12th, 2027 59275.37855344239
22 July 12th, 2027 59275.37855344458
23 October 11th, 2027 59275.37855344458
24 January 10th, 2028 59275.37855344458
25 April 10th, 2028 59275.37855344458
26 July 10th, 2028 59275.3785534445